In [21]:
%pip install -e git+https://github.com/cldf-datasets/wals.git@v2020#egg=cldfbench_wals --quiet

Note: you may need to restart the kernel to use updated packages.


In [22]:
from cldfbench import get_dataset
import pandas as pd

In [23]:
wals = get_dataset("wals").cldf_reader()

In [24]:
def take_n_from(it, n):
    return (x[1] for x in zip(range(n), it))

In [25]:
# languages = take_n_from(wals.iter_rows("LanguageTable"), 5)
lang_df = pd.DataFrame(wals.iter_rows("LanguageTable"))
lang_df.head(5)

,ID,Name,Macroarea,Latitude,Longitude,Glottocode,ISO639P3code,Family,Subfamily,Genus,ISO_codes,Samples_100,Samples_200
0,aab,Arapesh (Abu),None,-3.45,142.95,None,None,Torricelli,None,Kombio-Arapesh,[],False,False
1,aar,Aari,None,6,36.5833333333,aari1239,aiw,Afro-Asiatic,Omotic,South Omotic,[aiw],False,False
2,aba,Abau,None,-4,141.25,abau1245,aau,Sepik,None,Upper Sepik,[aau],False,False
3,abb,Arabic (Chadian),None,13.8333333333,20.8333333333,chad1249,shu,Afro-Asiatic,None,Semitic,[shu],False,False
4,abd,Abidji,None,5.66666666667,-4.58333333333,abid1235,abi,Niger-Congo,None,Kwa,[abi],False,False


In [26]:
# parameters = take_n_from(wals.iter_rows("ParameterTable"), 5)
param_df = pd.DataFrame(wals.iter_rows("ParameterTable"))["Area", "Chapter", "ID", "Name"].set_index(
    ["Area", "Chapter"]
)
param_df.head(5)

ID  \
Area      Chapter                                   
Phonology Consonant Inventories                1A   
          Vowel Quality Inventories            2A   
          Consonant-Vowel Ratio                3A   
          Voicing in Plosives and Fricatives   4A   
          Voicing and Gaps in Plosive Systems  5A   

                                                                              Name  \
Area      Chapter                                                                    
Phonology Consonant Inventories                              Consonant Inventories   
          Vowel Quality Inventories                      Vowel Quality Inventories   
          Consonant-Vowel Ratio                              Consonant-Vowel Ratio   
          Voicing in Plosives and Fricatives    Voicing in Plosives and Fricatives   
          Voicing and Gaps in Plosive Systems  Voicing and Gaps in Plosive Systems   

                                              Description Contributor_ID  
Area      Chapter                                                         
Phonology Consonant Inventories                      None   [maddiesoni]  
          Vowel Quality Inventories                  None   [maddiesoni]  
          Consonant-Vowel Ratio                      None   [maddiesoni]  
          Voicing in Plosives and Fricatives         None   [maddiesoni]  
          Voicing and Gaps in Plosive Systems        None   [maddiesoni]

In [31]:
param_df.groupby(level=0).head(5)

ID  \
Area               Chapter                                                
Phonology          Consonant Inventories                             1A   
                   Vowel Quality Inventories                         2A   
                   Consonant-Vowel Ratio                             3A   
                   Voicing in Plosives and Fricatives                4A   
                   Voicing and Gaps in Plosive Systems               5A   
Morphology         Fusion of Selected Inflectional Formatives       20A   
                   Exponence of Selected Inflectional Formatives    21A   
                   Exponence of Selected Inflectional Formatives    21B   
                   Inflectional Synthesis of the Verb               22A   
                   Locus of Marking in the Clause                   23A   
Nominal Categories Number of Genders                                30A   
                   Sex-based and Non-sex-based Gender Systems       31A   
                   Systems of Gender Assignment                     32A   
                   Coding of Nominal Plurality                      33A   
                   Occurrence of Nominal Plurality                  34A   
Nominal Syntax     Obligatory Possessive Inflection                 58A   
                   Obligatory Possessive Inflection                 58B   
                   Possessive Classification                        59A   
                   Genitives, Adjectives and Relative Clauses       60A   
                   Adjectives without Nouns                         61A   
Verbal Categories  Perfective/Imperfective Aspect                   65A   
                   The Past Tense                                   66A   
                   The Future Tense                                 67A   
                   The Perfect                                      68A   
                   Position of Tense-Aspect Affixes                 69A   
Word Order         Order of Subject, Object and Verb                81A   
                   Order of Subject, Object and Verb                81B   
                   Order of Subject and Verb                        82A   
                   Order of Object and Verb                         83A   
                   Order of Object, Oblique, and Verb               84A   
Simple Clauses     Alignment of Case Marking of Full Noun Phrases   98A   
                   Alignment of Case Marking of Pronouns            99A   
                   Alignment of Verbal Person Marking              100A   
                   Expression of Pronominal Subjects               101A   
                   Verbal Person Marking                           102A   
Complex Sentences  Relativization on Subjects                      122A   
                   Relativization on Obliques                      123A   
                   'Want' Complement Subjects                      124A   
                   Purpose Clauses                                 125A   
                   'When' Clauses                                  126A   
Lexicon            Hand and Arm                                    129A   
                   Finger and Hand                                 130A   
                   Finger and Hand                                 130B   
                   Numeral Bases                                   131A   
                   Number of Non-Derived Basic Colour Categories   132A   
Sign Languages     Irregular Negatives in Sign Languages           139A   
                   Question Particles in Sign Languages            140A   
Other              Writing Systems                                 141A   
                   Para-Linguistic Usages of Clicks                142A   

                                                                                                                Name  \
Area               Chapter                                                                                             
Pho

In [28]:
# codes = take_n_from(wals.iter_rows("CodeTable"), 5)
code_df = pd.DataFrame(wals.iter_rows("CodeTable"))
code_df.head(5)

,ID,Parameter_ID,Name,Description,Number,icon
0,1A-1,1A,Small,Small,1,c0000dd
1,1A-2,1A,Moderately small,Moderately small,2,c9999ff
2,1A-3,1A,Average,Average,3,cffffff
3,1A-4,1A,Moderately large,Moderately large,4,cff66ff
4,1A-5,1A,Large,Large,5,cdd0000


In [29]:
# values = take_n_from(wals.iter_rows("ValueTable"), 5)
value_df = pd.DataFrame(wals.iter_rows("ValueTable"))
value_df.head(5)

,ID,Language_ID,Parameter_ID,Value,Code_ID,Comment,Source,Example_ID
0,81A-aab,aab,81A,2,81A-2,None,[Nekitel-1985[94]],[]
1,82A-aab,aab,82A,1,82A-1,None,[Nekitel-1985[94]],[]
2,83A-aab,aab,83A,2,83A-2,None,[Nekitel-1985[94]],[]
3,87A-aab,aab,87A,2,87A-2,None,[Nekitel-1985[95]],[]
4,88A-aab,aab,88A,2,88A-2,None,[Nekitel-1985[95]],[]


In [30]:
# examples = take_n_from(wals.iter_rows("ExampleTable"), 5)
example_df = pd.DataFrame(wals.iter_rows("ExampleTable"))
example_df.head(5)

,ID,Language_ID,Primary_Text,Analyzed_Word,Gloss,Translated_Text,Meta_Language_ID,Comment
0,igt-1,mnd,tāmen tōu zìxíngchē,"[tāmen, tōu, zìxíngchē]","[3PL, steal, bicycle]",They steal bicycles.,None,None
1,igt-2,mnd,[wǒ gěi nǐ de] shū,"[[wǒ, gěi, nǐ, de], shū]","[[1SG, give, 2SG, LINK], book]",the book [that I gave you],None,None
2,igt-3,nbd,dımın ıdw ṍww(ı),"[dımın, ıdw, ṍww(ı)]","[ten, eight, two]",82,None,None
3,igt-4,geo,v-,[v-],[1 SG /1 PL],I we,None,None
4,igt-5,dre,troa traqa la nekönatr,"[troa, traqa, la, nekönatr]","[oblig, arrive, the, child]",the child must come,None,None


Take a look at: https://github.com/cldf/cldfviz/tree/main